# Moment tensor inversion of perforation shots

Note: results in this Notebook are used in the Geophysics paper 2023. They concern perforation shots in the monitor well using negative offsets (untimulated part of the reservoir)

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy
import os

sep = seppy.sep()

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.3, xmin=-450, xmax=-200, grid=True, colorbar=False, curve=np.array([]), linestyle='--'):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offset (m)')
    plt.ylabel('Time (s)')
    if grid==True:
        plt.grid(color='w', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.show()

def wavelet(nt=101, dt=0.002, t0=0, sig1=0.003, sig2=0.02, derivative=True):
    """Build a time function that is the generalized Hubbert's pimple (or its derivative)"""
    
    it0=int(nt/2)
    t=(np.linspace(0,nt-1,nt)-it0)*dt
    a=(sig2/sig1)**(-sig2/(sig1+sig2)) + (sig2/sig1)**(sig1/(sig1+sig2))
    a=a*a
    if derivative==True:
        w=-a*(-np.exp((t0-t)/sig1)/sig1 + np.exp((t-t0)/sig2)/sig2)/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**3
    else:
        w=0.5*a/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**2
    
    return w

In [ ]:
# read data, Green's functions, mask, attributes

# first stage 2
# last stage 28
# nb of stages 20 (missing 8, 9, 12, 17, 22, 25, 26)
# no plug between stages 3 and 4, 25 and 26, 27 and 28, these pairs of stages are perforated at the same time
# there are 100 perfs in total (0 to 99)

axes, data = sep.read_file(datapath+"ch4_dirac.H.deriv2.bp")
wavelet_step = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dirac.H.deriv3.bp")
wavelet_pulse = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_das_raw_sp49.H")
praw = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_das_proc_sp49.H")
proc = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_das_proc_sp49.H.fx")
proc_fx = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_das_proc_sp49.H.fv")
proc_fv = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_das_proc_sp49.H.fk")
proc_fk = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_das.H.ss.bp")
pdata = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_mute_mask.H.ss")
pmute = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dxGnx.H.ss.bp")
pdxGnx = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dzGnz.H.ss.bp")
pdzGnz = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch4_dxGnz.H.ss.bp")
pdxGnz = data.reshape(axes.n,order='F').T

pattr = np.loadtxt(datapath+"ch4_sattr.txt")


dt=axes.d[0]
dx=axes.d[1]
nt=axes.n[0]
ntr=axes.n[1]
ns=axes.n[2]

In [ ]:
# QC plots
sp=49

plotseis(pdata[sp])
plotseis(pdxGnx[sp])
plotseis(pdzGnz[sp])
plotseis(pdxGnz[sp])

## Part I: Preliminary MT inversion and time function calibration

To transform data from 3D to 2D, it is convolved with the filter $\sqrt{2\pi cr}\frac{H(t)}{\sqrt{t}}$, where $c=4.5$ km/s, $r=0.2$ km, $H(t)$ is the Heaviside function.
A single moment rate time function is proposed for the moment tensors, which is the time derivative of the generalized Hubbert's Pimple (Claerbout and Ronen, SEP 184):
$$ \dot{w}(t) = -a\frac{\Big( -\frac{1}{\sigma_1} e^{(t_0-t)/\sigma_1} + \frac{1}{\sigma_2} e^{(t-t_0)/\sigma_2} \Big)}{\Big( e^{(t_0-t)/\sigma_1} +  e^{(t-t_0)/\sigma_2} \Big)^3} $$
In seismology, moment rate function is normalized such that $\int_{-\infty}^{\infty}\dot{w}(t)dt=1$. Here $\dot{w}$ cannot be normalized the same way because it integrates to zero (no DC component). Therefore it is normalized such as $\int_{-\infty}^{t_1}\dot{w}(t)dt=\frac{1}{2}$, where
$$t_1 = t_0 + \frac{\sigma_1 \sigma_2 \log(\frac{\sigma_2}{\sigma_1})}{\sigma_1+\sigma_2},$$
which leads to
$$a = \Big( (\frac{\sigma_2}{\sigma_1})^{-\frac{\sigma_2}{\sigma_1+\sigma_2}} + (\frac{\sigma_2}{\sigma_1})^{\frac{\sigma_1}{\sigma_1+\sigma_2}} \Big)^2$$

In [ ]:
# build example time function corresponding to Moment function and Moment rate function

sig1=0.003
sig2=0.02
t0=0
it0=int(nt/2)

w0 = wavelet(nt=nt, dt=dt, t0=t0, sig1=sig1, sig2=sig2, derivative=False)
w = wavelet(nt=nt, dt=dt, t0=t0, sig1=sig1, sig2=sig2, derivative=True)

plt.figure(figsize=(6.66,2),dpi=300)
plt.subplot(1,2,1)
plt.plot(np.linspace(-it0*dt,it0*dt,nt),w0,label="HP")
plt.xlabel("Time (s)")
# plt.ylabel("Amplitude")
plt.ylabel("Amplitude (adimensional)")
plt.xlim([-0.1,0.1])
# plt.title("Moment time function (dimensionless)")
plt.title("a)",loc='left')
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.subplot(1,2,2)
plt.plot(np.linspace(-it0*dt,it0*dt,nt),w,label="Derivative of HP")
plt.xlabel("Time (s)")
plt.xlim([-0.1,0.1])
plt.ylabel(r"Amplitude ($s^{-1}$)")
# plt.title(r"Moment rate function ($s^{-1}$)")
plt.title("b)",loc='left')
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()

plt.savefig(figpath+'wavelets.png',bbox_inches='tight',format='png')

In [ ]:
# Figure to show the difference in the far-field between the two extreme cases for the moment function: step function vs impulse
# the bandpass filter is included

plt.figure(figsize=(6.66,2),dpi=300)

plt.plot(np.linspace(0,(nt-1)*dt,nt),wavelet_step[0]/np.amax(wavelet_step[0]),'k',label=r"$\sigma_1 \rightarrow 0,~ \sigma_2 \rightarrow \infty$")
plt.plot(np.linspace(0,(nt-1)*dt,nt),wavelet_pulse[0]/np.amax(wavelet_pulse[0]),'r',label=r"$\sigma_1 \rightarrow 0,~ \sigma_2 \rightarrow 0$")
plt.plot(np.linspace(0,(nt-1)*dt,nt),ndimage.shift(wavelet_pulse[0],1.5)/np.amax(wavelet_pulse[0]),'r--',label=r"$\sigma_1 \rightarrow 0,~ \sigma_2 \rightarrow 0$ shifted")
plt.xlabel("Time (s)")
plt.ylabel("Normalized amplitude")
plt.xlim([0,0.05])
plt.legend(loc='upper right',prop={'size': 6})
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.savefig(figpath+'wavelets_qc.png',bbox_inches='tight',format='png')

In [ ]:
# for every shot
#    scan sigma1, sigma2>=sigma1, origin time
#    convolve GFs with the time function
#    build matrix G and invert the MT
#    find the smallest normalized functional
#    deduce the optimal origin time, sigma1, sigma2

origin_t=np.zeros(ns)
sigma1=np.zeros(ns)
sigma2=np.zeros(ns)
nfunc=999*np.ones(ns)
mt=np.zeros((ns,2))

# time shift boundaries in sec
otmin=-0.004
otmax=0.002
otnum=7

# sigma boundaries is sec
sigmin=dt/2
signum1=15
signum2=25

# full functional
nfunc_all=np.zeros((ns,otnum*signum1*signum2))

# predicted data
psyn=np.copy(pdata)

In [ ]:
spmin=0
spmax=100

# loop over shots
for s in range(spmin,spmax):
    
    print("Processing shot %d" %s)
    
    # data norm
    d=np.reshape(pdata[s,:,:],(ntr*nt))
    dnorm=np.linalg.norm(d)
    
    # make copies of GFs
    pdxGnx_w=np.copy(pdxGnx[s,:,:])
    pdzGnz_w=np.copy(pdzGnz[s,:,:])
    
    # full functional for the given shot
    func=np.zeros(0)
    
    # loop over sigmas
    for sig1 in np.linspace(sigmin,signum1*sigmin,signum1):
        for sig2 in np.linspace(sigmin,signum2*sigmin,signum2):
            
            if (sig2>=sig1):

                # loop over time origin
                for shift in np.linspace(otmin,otmax,otnum):

                    # build the shifted time function
                    w = wavelet(nt=nt, dt=dt, t0=shift, sig1=sig1, sig2=sig2, derivative=True)

                    # convolve GF with the time functions
                    for itr in range(ntr):
                        pdxGnx_w[itr,:]=dt*np.convolve(pdxGnx[s,itr,:],w,mode='same')
                        pdzGnz_w[itr,:]=dt*np.convolve(pdzGnz[s,itr,:],w,mode='same')


                    # form the matrix G
                    G=np.vstack((np.reshape(pdxGnx_w,(nt*ntr)),np.reshape(pdzGnz_w,(nt*ntr))))
                    G=np.transpose(G)

                    # Invert the MT using the pseudo inverse of G
                    m=np.matmul(np.linalg.pinv(G),d)

                    # Compute predicted data
                    d_hat=np.matmul(G,m)

                    # Compute the normalized functional
                    f=np.linalg.norm(d_hat-d)/dnorm
                    func=np.append(func,f)

                    # check if the normalized residual is the smallest so far
                    if (f<nfunc[s]):
                        nfunc[s]=f
                        mt[s,:]=m
                        origin_t[s]=shift
                        sigma1[s]=sig1
                        sigma2[s]=sig2
                        psyn[s,:,:]=np.reshape(d_hat,(ntr,nt))
            else:
                # loop over time origin
                for shift in np.linspace(otmin,otmax,otnum):
                    func=np.append(func,999)
                    
    nfunc_all[s] = func

In [ ]:
# save the outputs
np.save(datapath+'origin_t',origin_t)
np.save(datapath+'sigma1',sigma1)
np.save(datapath+'sigma2',sigma2)
np.save(datapath+'mt',mt)
np.save(datapath+'nfunc',nfunc)
np.save(datapath+'nfunc_all',nfunc_all)
np.save(datapath+'syn',psyn)

In [ ]:
# load the outputs
origin_t=np.load(datapath+'origin_t.npy')
sigma1=np.load(datapath+'sigma1.npy')
sigma2=np.load(datapath+'sigma2.npy')
mt=np.load(datapath+'mt.npy')
nfunc=np.load(datapath+'nfunc.npy')
nfunc_all=np.load(datapath+'nfunc_all.npy')
psyn=np.load(datapath+'syn.npy')

In [ ]:
sp=49
ff=nfunc_all[sp].reshape((signum1,signum2,otnum))
vmin=np.min(ff)
#vmax=np.max(ff)
vmax=1.1
plt.figure(figsize=(6.66,4),dpi=300)
for i in range(1,otnum):
    plt.subplot(2,3,i)
    plt.imshow(ff[:,:,i],cmap='coolwarm',extent=[1000*sigmin,1000*signum2*sigmin,1000*signum1*sigmin,1000*sigmin],vmin=vmin,vmax=vmax)
    #plt.colorbar(fraction=0.04)
    ot=otmin+i*(otmax-otmin)/(otnum-1)
    plt.title(r"$t_0$=%.0f ms" %(1000*ot))
    if (abs(origin_t[sp]-ot)<0.00001):
        plt.scatter(1000*sigma2[sp],1000*sigma1[sp],c='w',s=50)        
    plt.gca().invert_yaxis()
    plt.gca().tick_params(axis='both', which='major', labelsize=6)
    if i>3:
        plt.xlabel(r'$\sigma_2$ (ms)')
    else:
        plt.gca().set_xticklabels([])
    if i in [1, 4]:
        plt.ylabel(r'$\sigma_1$ (ms)')
    else:
        plt.gca().set_yticklabels([])
    
#     plt.gca().set(aspect='equal')
#     plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
#     plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout(pad=0., w_pad=0., h_pad=-6.)
plt.savefig(figpath+'func_global.png',bbox_inches='tight',format='png')

In [ ]:
spmax=100
plt.figure(figsize=(6.66,4),dpi=300)

plt.subplot(2,2,1)
bins=1000*np.linspace(sigmin,6*sigmin,6)-0.5
plt.hist(1000*sigma1[:spmax],bins=bins)
plt.xlabel(r'Optimal $\sigma_1$ (ms)')
plt.ylabel("Shot count",fontsize=12)
plt.xticks([2,3,4,5])
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.subplot(2,2,2)
bins=np.linspace(4,25,22)-0.5
plt.hist(1000*sigma2[:spmax],bins=bins)
plt.xlabel(r'Optimal $\sigma_2$ (ms)')
plt.xticks()
plt.xticks(np.arange(4,signum2+1))
#plt.ylabel("Shot count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.subplot(2,2,4)
bins=np.linspace(-4,2,7)-0.5
plt.hist(1000*origin_t[:spmax],bins=bins)
plt.xlabel(r'Optimal time shift (ms)')
plt.ylabel("Shot count",fontsize=12)
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()

plt.savefig(figpath+'hist_s1s2t.png',bbox_inches='tight',format='png')

print("Median sigma1 = %f sec" %np.median(sigma1))
print("Median sigma2 = %f sec" %np.median(sigma2))
print("Median t0 = %f sec" %np.median(origin_t))

In [ ]:
spmax=100

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.hist(1000*mt[:spmax,0],bins=10)
plt.xlabel(r"$M_{11}$ (scaled N.m)",fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.ylim([0, 40])

plt.subplot(1,2,2)
plt.hist(1000*mt[:spmax,1],bins=10)
plt.xlabel(r"$M_{33}$ (scaled N.m)",fontsize=12)
# plt.ylabel("Shot count",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 40])

plt.tight_layout()

In [ ]:
sp=49
p=1
vmax=p*np.amax(pdxGnx[sp])

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(pdxGnx[sp]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $M_{11}$',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(pdzGnz[sp]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
# plt.xlabel('Offset (m)')
# plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("DAS data - shot %d" %sp)
# plt.title("DAS observed")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $M_{33}$',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(pdxGnz[sp]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("Modeled data - shot %d" %sp)
# plt.title("DAS modeled")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) $M_{13}$',loc='left')

# plt.tight_layout()
plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'GF.png',bbox_inches='tight',format='png')

In [ ]:
sp=49
p=0.99
vmax=p*np.amax(pdata[sp])

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(praw[350:601,0:601]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-50*vmax,vmax=50*vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a)',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(pdata[sp]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
# plt.xlabel('Offset (m)')
# plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("DAS data - shot %d" %sp)
# plt.title("DAS observed")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b)',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn[sp]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("Modeled data - shot %d" %sp)
# plt.title("DAS modeled")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c)',loc='left')

plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'das_obs_syn.png',bbox_inches='tight',format='png')

In [ ]:
print("Median of time shifts from original pass is %f" %np.median(origin_t))

## Part II: Data calibration

### Calibrating the amplitude decay with offset using macro shots

In [ ]:
# stack data and synthetics to create macro shot gathers
macro_data=np.sum(pdata,axis=0)/ns
macro_syn=np.sum(psyn,axis=0)/ns

In [ ]:
p=0.99
vmax=p*np.amax(macro_data)

fig=plt.figure(figsize=(6.66, 3),dpi=300)
ax = plt.subplot(1,2,1)
plt.imshow(np.transpose(macro_data),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title("Observed super shot")
plt.gca().tick_params(axis='both', which='major', labelsize=6)

ax = plt.subplot(1,2,2)
plt.imshow(np.transpose(macro_syn),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title("Modeled super shot")
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()

plt.savefig(figpath+'super_shot.png',bbox_inches='tight',format='png')

In [ ]:
# compute the envelop
macro_data_env = np.absolute(signal.hilbert(macro_data,axis=1))
macro_syn_env = np.absolute(signal.hilbert(macro_syn,axis=1))

# pick the maximum
nx=macro_data.shape[0]
data_max=np.zeros(nx)
data_amax=np.zeros(nx,dtype=int)
syn_max=np.zeros(nx)
syn_amax=np.zeros(nx,dtype=int)

for i in range(nx):
    data_amax[i] = int(np.argmax(macro_data_env[i,:]))
    syn_amax[i] = int(np.argmax(macro_syn_env[i,:]))
    data_max[i] = macro_data_env[i,data_amax[i]]
    syn_max[i] = macro_syn_env[i,syn_amax[i]]

In [ ]:
p=0.99
vmax=p*np.amax(macro_data_env)

fig=plt.figure(figsize=(6.66, 3),dpi=300)
ax = plt.subplot(1,2,1)
plt.imshow(np.transpose(macro_data_env),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=0,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='r', linestyle='-', linewidth=0.2)
plt.title("Observed super envelop")
plt.plot(np.linspace(-450,-450+(nx-1),nx),dt*data_amax)
plt.gca().tick_params(axis='both', which='major', labelsize=6)

ax = plt.subplot(1,2,2)
plt.imshow(np.transpose(macro_syn_env),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=0,vmax=vmax)
plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='r', linestyle='-', linewidth=0.2)
plt.title("Modeled super envelop")
plt.plot(np.linspace(-450,-450+(nx-1),nx),dt*syn_amax)
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()

plt.savefig(figpath+'super_envelop.png',bbox_inches='tight',format='png')

In [ ]:
p1 = np.polyfit(np.log10(np.linspace(200,450,251)),np.log10(np.flip(data_max)),deg=1)
p2 = np.polyfit(np.log10(np.linspace(200,450,251)),np.log10(np.flip(syn_max)),deg=1)

fig=plt.figure(figsize=(3.33, 2),dpi=300)

plt.scatter(np.log10(np.linspace(200,450,251)),np.log10(np.flip(data_max)),color='b',s=0.1,label="Max envelop - observed")
plt.scatter(np.log10(np.linspace(200,450,251)),np.log10(np.flip(syn_max)),color='r',s=0.1,label="Max envelop - modeled")
plt.plot(np.log10(np.linspace(200,450,251)),np.log10(np.linspace(200,450,251))*p1[0]+p1[1],'b--',linewidth=1,label="Fitted line - observed")
plt.plot(np.log10(np.linspace(200,450,251)),np.log10(np.linspace(200,450,251))*p2[0]+p2[1],'r--',linewidth=1,label="Fitted line - modeled")
plt.xlabel(r"$\log_{10}$(|Offset|)",fontsize=12)
plt.ylabel(r"$\log_{10}$(Amplitude)",fontsize=12)
#plt.title("Amplitude decay with offset")
plt.legend(loc='upper right',prop={'size': 6})
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.savefig(figpath+'amplitude_decay.png',bbox_inches='tight',format='png')

print("The fitted line for DAS data has a slope p = %f" %p1[0])
print("The fitted line for synthetic data has a slope q = %f" %p2[0])

#### Compute weighting function
Based on amplitude decay vs offset obtained from the envelop of field and synthetic super shots, a weighting function is deduced, to be applied to field data before the MT inversion:

$$f(x) = (\frac{x}{x_0})^{q-p}$$

where $x=\text{offset}$, $x_0=200 \text{ m}$, $p=-1.41$, and $q=-1.15$.

In [ ]:
weights=(np.linspace(200,451,251)/200)**(p2[0]-p1[0])
weights=np.flip(weights)
weights=np.reshape(weights,(weights.shape[0],1))

### Trace alignment and selection based on max correlation with synthetics

In [ ]:
# for every shot
#    apply amplitude correction
#    compute correlation with synthetic
#    shift to maximum correlation
#    zero traces with NCC < percentile

pdata2=np.copy(pdata)
psyn2=np.copy(psyn)
pdata2_flag=np.zeros((ns,ntr))
pdata2_flag[:,:]=1

tshifts=np.zeros((ns,ntr))
ncctr=np.zeros((ns,ntr))

spmin=0
spmax=100

lagmin=-3
lagmax=3

percentile=35

for s in range(spmin,spmax):
            
    for itr in range(ntr):
        
        # apply amplitude correction
        pdata2[s,itr,:] *= weights[itr]
        
        # 1d correlation
        corr=np.correlate(pdata2[s,itr,:],psyn[s,itr,:],mode='full')
        
        # maximum correlation
        ind_max=np.argmax(corr)
        
        # deduce time shift
        ts=ind_max-(nt-1)
        tshifts[s,itr]=-ts
        
        # set to zero if outside the lag range
        if ts<lagmin or ts>lagmax:
            ts=0
            
        # apply the shifts
        pdata2[s,itr,:]=ndimage.shift(pdata2[s,itr,:],-ts)
        
        # calculate the normalized cross-correlation coefficients NCC
        ncctr[s,itr] = np.dot(pdata2[s,itr,:],psyn2[s,itr,:])/(np.linalg.norm(pdata2[s,itr,:])*np.linalg.norm(psyn2[s,itr,:]))
        
    # calculate the specified percentile of NCC
    perc=np.percentile(ncctr[s,:],percentile)
    
    # zero out traces with NCC < specified percentile
    tozero = np.argwhere(ncctr[s,:]<perc)
    pdata2[s,tozero,:]=0
    psyn2[s,tozero,:]=0
    pdata2_flag[s,tozero]=0

In [ ]:
sp=49
p=0.9
vmax=p*np.amax(pdata2[sp])
fig, ax = plt.subplots(1,1,figsize=(3, 3),dpi=300)
im=ax.imshow(np.transpose(pdata2[sp]),interpolation='none',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
ax.set(xlabel='Offset (m)')
ax.set(ylabel='Time (s)')
#cbar=fig.colorbar(im, ax=ax,format='%.'+str(1)+'f',aspect=20)
ax.grid(color='w', linestyle='-', linewidth=0.2)
plt.title("Decimated gather - shot %d" %sp)
#plt.show()
#plt.savefig(figpath+'decimated_gather.png',bbox_inches='tight',format='png')

In [ ]:
# remove traces having all zeroes from data and GFs
pdata3 = pdata2[~np.all(pdata2 == 0, axis=2),:]
pdata3 = np.reshape(pdata3,(ns,int(pdata3.size/(ns*nt)),nt))

pdxGnx3=pdxGnx[~np.all(pdata2 == 0, axis=2),:]
pdzGnz3=pdzGnz[~np.all(pdata2 == 0, axis=2),:]
pdxGnz3=pdxGnz[~np.all(pdata2 == 0, axis=2),:]
pdxGnx3 = np.reshape(pdxGnx3,(ns,int(pdata3.size/(ns*nt)),nt))
pdzGnz3 = np.reshape(pdzGnz3,(ns,int(pdata3.size/(ns*nt)),nt))
pdxGnz3 = np.reshape(pdxGnz3,(ns,int(pdata3.size/(ns*nt)),nt))

## Part III: Final MT inversion

### Extract stiffness components at source locations

In [ ]:
# read VTI model and source-receiver coordinates (in the rotated system)

axes, data = sep.read_file(datapath+"ch4_elasticModelVTI1.H")
pmod = data.reshape(axes.n,order='F').T

sxz = pattr[:,[0,19,21]]
sxz[:,1:] *= 1e-5 # convert from cm to km
srxz=np.loadtxt(datapath+'ch4_srcoord.txt')

Zn = axes.n[0]
Zo = axes.o[0]
Zd = axes.d[0]

Xn = axes.n[1]
Xo = axes.o[1]
Xd = axes.d[1]

assert(ns == sxz.shape[0])

In [ ]:
# extract vp, vs, rho, delta, epsilon at source locations
vp=np.zeros(ns)
vs=np.zeros(ns)
rho=np.zeros(ns)
delta=np.zeros(ns)
epsilon=np.zeros(ns)

for s in range(ns):
    
    ix = int((sxz[s,1] - Xo)/Xd)
    iz = int((sxz[s,2] - Zo)/Zd)
    wx = (sxz[s,1] - ix*Xd - Xo)/Xd
    wz = (sxz[s,2] - iz*Zd - Zo)/Zd
    
    vp[s] = (1-wx)*(1-wz)*pmod[0,ix,iz] + wx*(1-wz)*pmod[0,ix+1,iz] + (1-wx)*wz*pmod[0,ix,iz+1] + wx*wz*pmod[0,ix+1,iz+1]
    vs[s] = (1-wx)*(1-wz)*pmod[1,ix,iz] + wx*(1-wz)*pmod[1,ix+1,iz] + (1-wx)*wz*pmod[1,ix,iz+1] + wx*wz*pmod[1,ix+1,iz+1]
    rho[s] = (1-wx)*(1-wz)*pmod[2,ix,iz] + wx*(1-wz)*pmod[2,ix+1,iz] + (1-wx)*wz*pmod[2,ix,iz+1] + wx*wz*pmod[2,ix+1,iz+1]
    delta[s] = (1-wx)*(1-wz)*pmod[3,ix,iz] + wx*(1-wz)*pmod[3,ix+1,iz] + (1-wx)*wz*pmod[3,ix,iz+1] + wx*wz*pmod[3,ix+1,iz+1]
    epsilon[s] = (1-wx)*(1-wz)*pmod[4,ix,iz] + wx*(1-wz)*pmod[4,ix+1,iz] + (1-wx)*wz*pmod[4,ix,iz+1] + wx*wz*pmod[4,ix+1,iz+1]

In [ ]:
# deduce C33, C11, C55, C66, C13, lambda, mu
gamma=0.53 # provisional value obtained from (roughly) comparing Vs_fast horizontal (2.3 km/s) and Vs_slow vertical (1.6 km/s) logs ; 

c33 = rho*vp*vp
c11 = (1+2*epsilon)*c33
c55 = rho*vs*vs
c66 = (1+2*gamma)*c55
c13 = np.sqrt(2*c33*(c33-c55)*delta + np.square(c33-c55)) - c55                   
mu = c55
lam = c33 - 2*mu

In [ ]:
nx1=0
nz1=550
nr1=49000
plt.figure(figsize=(6.66,2),dpi=300)
plt.imshow(np.transpose(pmod[0,nx1:,nz1:]),cmap='jet',extent=[Xo+nx1*Xd,Xo+nx1*Xd+(Xn-nx1-1)*Xd,Zo+nz1*Zd+(Zn-nz1-1)*Zd,Zo+nz1*Zd],vmin=2.5,vmax=6.0,interpolation='bilinear', aspect='auto')
# plt.colorbar().set_label(label="km/s",size=16)
plt.colorbar(format="%d",pad=0.01,label="km/s")
plt.scatter(srxz[:-nr1,3],srxz[:-nr1,4],s=0.5,c='y',marker='.')
plt.scatter(sxz[:,1],sxz[:,2],s=10,c='r',marker='x')
plt.xlabel(r'X (km)')
plt.ylabel(r'Depth (km)')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
#plt.title(r'$V_p$ model with sources-receivers location')
# plt.gca().set_xticks([])
# plt.gca().set_yticks([])
#plt.gca().set(aspect='equal')
#plt.tight_layout()
plt.savefig(figpath+'vp.png',bbox_inches='tight',format='png')

### MT inversion using pseudo-inverse (least-squares solution)

In [ ]:
# for every shot
#    convolve GFs with time functions, fixed origin time and fixed sigma1 sigma2 (median values)
#    build matrix G and invert the scalers of structured (and unstructured) MTs

ntr3=pdata3.shape[1]

nfunc3=999*np.ones(ns)
scalers=np.zeros((ns,2))
scalersb=np.zeros((ns,3)) # for when Mxz != 0

# median sigmas is sec
sigma1_med=np.median(sigma1)
sigma2_med=np.median(sigma2)

# median time shifts
origin_t_med=np.median(origin_t)

# condition number of the matrix G
cond=np.zeros(ns)
condb=np.zeros(ns)

# matrix G'G
GTG=np.zeros((ns,2,2))

# predicted data
psyn3=np.copy(pdata3)
psyn3b=np.copy(pdata3)

spmin=0
spmax=100

# loop over shots
for s in range(spmin,spmax):
        
    # data norm
    d=np.reshape(pdata3[s,:,:],(ntr3*nt))
    dnorm=np.linalg.norm(d)
    
    # make copies of GFs
    pdxGnx_w=np.copy(pdxGnx3[s,:,:])
    pdzGnz_w=np.copy(pdzGnz3[s,:,:])
    pdxGnz_w=np.copy(pdxGnz3[s,:,:])
    
    # build the time function
    w = wavelet(nt=nt, dt=dt, t0=origin_t_med, sig1=sigma1_med, sig2=sigma2_med, derivative=True)
    
    # convolve GF with the time functions
    for itr in range(ntr3):
        pdxGnx_w[itr,:]=dt*np.convolve(pdxGnx3[s,itr,:],w,mode='same')
        pdzGnz_w[itr,:]=dt*np.convolve(pdzGnz3[s,itr,:],w,mode='same')
        pdxGnz_w[itr,:]=dt*np.convolve(pdxGnz3[s,itr,:],w,mode='same')

            
    # form the matrix G
    G=np.vstack((np.reshape(pdxGnx_w,(nt*ntr3)),np.reshape(pdzGnz_w,(nt*ntr3))))
    Gb=np.vstack((np.reshape(pdxGnx_w,(nt*ntr3)),np.reshape(pdzGnz_w,(nt*ntr3)),np.reshape(pdxGnz_w,(nt*ntr3))))
    G=np.transpose(G)
    Gb=np.transpose(Gb)
                                
    # Find the MT using the pseudo inverse of G (least-squares solution)
    m=np.matmul(np.linalg.pinv(G),d)
    mb=np.matmul(np.linalg.pinv(Gb),d)
                
    # Compute predicted data
    d_hat=np.matmul(G,m)
    d_hatb=np.matmul(Gb,mb)

    # Compute the normalized functional
    f=np.linalg.norm(d_hat-d)/dnorm
    
    scalers[s,:]=m
    scalersb[s,:]=mb
    psyn3[s,:,:]=np.reshape(d_hat,(ntr3,nt))
    psyn3b[s,:,:]=np.reshape(d_hatb,(ntr3,nt))
    nfunc3[s]=f
    cond[s]=np.linalg.cond(G)
    condb[s]=np.linalg.cond(Gb)
    GTG[s]=np.matmul(np.transpose(G),G)

### Plot results, QC and analyze

In [ ]:
print("Median values for M11 and M33 for structured inversion are %f %f" %(np.median(scalers[:,0]),np.median(scalers[:,1])) )
print("Median values for M11, M33, and M13 for unstructured inversion are %f %f %f" %(np.median(scalersb[:,0]),np.median(scalersb[:,1]),np.median(scalersb[:,2])) )

In [ ]:
spmax=100

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.hist(1000*scalers[:spmax,0],bins=10)
plt.xlabel(r"$M_{11}$ (scaled N.m)",fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.ylim([0, 38])

plt.subplot(1,2,2)
plt.hist(1000*scalers[:spmax,1],bins=10)
plt.xlabel(r"$M_{33}$ (scaled N.m)",fontsize=12)
# plt.ylabel("Shot count",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 38])

plt.tight_layout()

plt.savefig(figpath+'mt_histogram.png',bbox_inches='tight',format='png')

In [ ]:
spmax=100

x=1000*scalers[:spmax,0]
y=1000*scalers[:spmax,1]

fig = plt.figure(figsize=(6.66, 4),dpi=300)
gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)

ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

ax_histx.set_ylabel("Shot count",fontsize=12)
ax_histy.set_xlabel("Shot count",fontsize=12)

ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)

im = ax.scatter(x, y, marker='x',c=np.sqrt(x**2+y**2),cmap='hot_r',s=50)
ax.set_xlabel(r"$M_{11}$ (scaled N.m)",fontsize=12)
ax.set_ylabel(r"$M_{33}$ (scaled N.m)",fontsize=12)
ax.set_facecolor('0.8')
plt.gca().tick_params(axis='both', which='major', labelsize=6)

# fig.colorbar(im, ax=ax, orientation="horizontal", pad=0.2)

dummy = ax_histx.hist(x, bins=10)
dummy = ax_histy.hist(y, bins=10, orientation='horizontal')

plt.savefig(figpath+'mt_scatter_histogram.png',bbox_inches='tight',format='png')

In [ ]:
spmax=100
plt.figure(figsize=(6.66,3.33),dpi=300)
plt.scatter(np.linspace(0,99,spmax),(c33[:spmax]+c13[:spmax])/(c11[:spmax]-2*c66[:spmax]+c13[:spmax]),c='r',s=5,label="Cylindrical explosion")
plt.scatter(np.linspace(0,99,spmax),c13[:spmax]/(c11[:spmax] - c66[:spmax]),c='b',s=5,label="Vertical cylindrical opening")
plt.scatter(np.linspace(0,99,spmax),(c11[:spmax] - c66[:spmax])/(c11[:spmax] - c66[:spmax]),c='g',s=5,label="Horizontal cylindrical opening")
plt.scatter(np.linspace(0,99,spmax),c13[:spmax]/(c11[:spmax]),c='m',s=5,label="Tensile crack")
plt.scatter(np.linspace(0,99,spmax),scalers[:spmax,1]/scalers[:spmax,0],marker='x',c=1000*scalers[:spmax,0],cmap='hot_r',s=10,label="Estimated")
plt.xlabel("Shot number",fontsize=12)
plt.ylabel(r"$M_{33}/M_{11}$",fontsize=12)
plt.colorbar().set_label(label=r"$M_{11}$ (scaled N.m)",size=6)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
ax=plt.gca()
ax.set_facecolor('0.8')
handles,labels = ax.get_legend_handles_labels()
handles = [handles[0], handles[1], handles[2], handles[3], handles[4]]
labels = [labels[0], labels[1], labels[2], labels[3], labels[4]]
ax.legend(handles,labels,fontsize=6)
# plt.legend(fontsize=6,loc='upper right')
#plt.legend()
#plt.ylim([-1,6.5])
plt.savefig(figpath+'mt_ratio.png',bbox_inches='tight',format='png')

In [ ]:
spmax=100
fig, ax = plt.subplots(figsize=(6.66,3.33),dpi=300)
# cax = ax.scatter(perflist[:spmax,0],scalers[:spmax,0],c=perflist[:,1],cmap='jet',s=25)
cax = ax.scatter(pattr[:spmax,4],scalers[:spmax,1]/scalers[:spmax,0],marker='x',c=pattr[:spmax,27],cmap='jet',s=20)
plt.xlim([1, 29])
ax.locator_params(nbins=15, axis='x')
ax.set_facecolor('0.8')
plt.xlabel("Stage number",fontsize=12)
# plt.ylabel(r"$M_{xx}$ (arbitrary units)",fontsize=12)
plt.ylabel(r"$M_{33}/M_{11}$",fontsize=12)
cbar = fig.colorbar(cax, ticks=[1, 2, 3, 4, 5, 6])
cbar.set_label(label = "Shot number within a stage", size=6)
cbar = cbar.ax.set_yticklabels(['1', '2', '3', '4', '5', '6']) 
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.savefig(figpath+'mt_ratio_per_stage.png',bbox_inches='tight',format='png')

In [ ]:
%matplotlib inline
spmax=100

plt.figure(figsize=(3.33,3.33),dpi=300)
plt.hist(cond[:spmax],bins=10)
plt.xlabel("Condition number")
plt.ylabel("Shot count")
# plt.title("Histogram of condition number")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.savefig(figpath+'condition_number.png',bbox_inches='tight',format='png')

print("Smallest condition number = %f" %np.min(cond))
print("Largest condition number = %f" %np.max(cond))

In [ ]:
spmax=100

plt.figure(figsize=(3.33,3.33),dpi=300)
plt.hist(condb[:spmax],bins=10)
plt.xlabel("Condition number")
plt.ylabel("Shot count")
# plt.title("Histogram of condition number")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.savefig(figpath+'condition_number_2.png',bbox_inches='tight',format='png')

print("Smallest condition number = %f" %np.min(condb))
print("Largest condition number = %f" %np.max(condb))

### Save the scalar MTs, the time function, and the full MTs (scalar MT x time function)

In [ ]:
w0 = wavelet(nt=nt, dt=dt, t0=origin_t_med, sig1=sigma1_med, sig2=sigma2_med, derivative=True)

# shift so the first sample corresponds to -20 ms (as the GF's)
w = ndimage.shift(w0,-(nt-1)/2.0+0.02/dt)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.linspace(-0.02,(nt-1)*dt-0.02,nt),w,label="Derivative of HP")
plt.xlabel("Time (sec)")
plt.ylabel("Amplitude")
plt.xlim([-0.02,0.25])
plt.gca().grid(color='k', linestyle='-', linewidth=0.2)
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.3f'))

In [ ]:
pfull_time_func=np.zeros((3,100,nt))
for s in range(100):
    pfull_time_func[0,s,:]=scalers[s,0]*w
    pfull_time_func[1,s,:]=scalers[s,1]*w
    pfull_time_func[2,s,:]=0

In [ ]:
np.savetxt(datapath+"ch4_scaler_MTs.txt",scalers, fmt='%.4f')
sep.write_file(datapath+"ch4_time_func.H", np.transpose(w), ds=np.array([dt]), os=np.array([0]), dpath=datapath)
sep.write_file(datapath+"ch4_all_time_func.H", np.transpose(pfull_time_func), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)

### Compute synthetic data (GFs convolved with the time function and weighted by the MTs)

In [ ]:
# convolve GFs with the time function and weight by MT
psyn_all0 = np.copy(pdxGnx)
for s in range(psyn_all0.shape[0]):
    for itr in range(psyn_all0.shape[1]):
        temp=dt*np.convolve(scalers[s,0]*pdxGnx[s,itr,:]+scalers[s,1]*pdzGnz[s,itr,:],w0,mode='same')
        psyn_all0[s,itr,:]=temp[:]

In [ ]:
sp1=55
sp2=78
sp3=88
sp4=71
#sp4=57
p=0.9
vmax=p*np.amax(pdata[sp1])

fig=plt.figure(figsize=(8, 8),dpi=300)
plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata[sp1]*weights),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
#plt.xlabel('Offset (m)')
plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp1,scalers[sp1,0],scalers[sp1,1]), fontsize=8)

plt.subplot(2,2,2)
plt.imshow(np.transpose(pdata[sp2]*weights),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
#plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp2,scalers[sp2,0],scalers[sp2,1]), fontsize=8)

plt.subplot(2,2,3)
plt.imshow(np.transpose(pdata[sp3]*weights),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp3,scalers[sp3,0],scalers[sp3,1]), fontsize=8)

plt.subplot(2,2,4)
plt.imshow(np.transpose(pdata[sp4]*weights),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp4,scalers[sp4,0],scalers[sp4,1]), fontsize=8)

plt.tight_layout()

plt.savefig(figpath+'das_examples.png',bbox_inches='tight',format='png')

In [ ]:
sp1=55
sp2=78
sp3=88
sp4=71
p=0.9
vmax=p*np.amax(pdata[sp1])


fig=plt.figure(figsize=(8, 8),dpi=300)
plt.subplot(2,2,1)
plt.imshow(np.transpose(psyn_all0[sp1]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
#plt.xlabel('Offset (m)')
plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp1,scalers[sp1,0],scalers[sp1,1]), fontsize=8)

plt.subplot(2,2,2)
plt.imshow(np.transpose(psyn_all0[sp2]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
#plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp2,scalers[sp2,0],scalers[sp2,1]), fontsize=8)

plt.subplot(2,2,3)
plt.imshow(np.transpose(psyn_all0[sp3]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp3,scalers[sp3,0],scalers[sp3,1]), fontsize=8)

plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn_all0[sp4]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp4,scalers[sp4,0],scalers[sp4,1]), fontsize=8)

plt.tight_layout()

plt.savefig(figpath+'syn_examples.png',bbox_inches='tight',format='png')

In [ ]:
sp1=55
sp2=78
p=0.9
vmax1=p*np.amax(pdata[sp1])
vmax2=p*np.amax(pdata[sp1])

fig=plt.figure(figsize=(8, 8),dpi=300)
plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata[sp1]*weights),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax1,vmax=vmax1)
#plt.xlabel('Offset (m)')
plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp1,scalers[sp1,0],scalers[sp1,1]), fontsize=8)

plt.subplot(2,2,2)
plt.imshow(np.transpose(pdata[sp2]*weights),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax2,vmax=vmax2)
#plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.title(r"Shot #%d ; $M_{11}=$%.1E ; $M_{33}=$%.1E (scaled N.m)" %(sp2,scalers[sp2,0],scalers[sp2,1]), fontsize=8)

plt.subplot(2,2,3)
plt.imshow(np.transpose(psyn_all0[sp1]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax1,vmax=vmax1)
plt.xlabel('Offset (m)')
# plt.gca().set_xticklabels([])
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# plt.title(r"Shot #%d ; $M_{xx}=$%.1f ; $M_{zz}=$%.1f" %(sp1,scalers[sp1,0],scalers[sp1,1]))

plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn_all0[sp2]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax2,vmax=vmax2)
plt.xlabel('Offset (m)')
#plt.ylabel('Time (sec)')
# plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# plt.title(r"Shot #%d ; $M_{xx}=$%.1f ; $M_{zz}=$%.1f" %(sp2,scalers[sp2,0],scalers[sp2,1]))
plt.tight_layout()

plt.savefig(figpath+'das_syn_examples.png',bbox_inches='tight',format='png')

In [ ]:
sp=49
p=0.99
vmax=p*np.amax(weights*pdata[sp])

xarray=np.arange(-450,-200+1)*(1-pdata2_flag[sp,:])
xarray = xarray[(xarray != 0)]

temp=np.copy(pdata[sp]*weights)
for itr in range(ntr):
    ts=-tshifts[sp,itr]
    if ts<lagmin or ts>lagmax:
        ts=0
    temp[itr,:]=ndimage.shift(temp[itr,:],-ts)

fig=plt.figure(figsize=(6.66, 3.33),dpi=300)

ax = plt.subplot(1,2,1)
plt.imshow(np.transpose(temp),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.scatter(xarray,0.02*np.ones((xarray.shape[0])),s=1,c='r')
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a)',loc='left')

ax = plt.subplot(1,2,2)
plt.imshow(np.transpose(psyn_all0[sp]),interpolation='sinc',aspect="auto",extent=[-450,-200,0.3,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.scatter(xarray,0.02*np.ones((xarray.shape[0])),s=1,c='r')
plt.xlabel('Offset (m)')
# plt.ylabel('Time (s)')
# plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("DAS data - shot %d" %sp)
# plt.title("DAS observed")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b)',loc='left')


plt.tight_layout(pad=0., w_pad=-0.)

plt.savefig(figpath+'das_obs_syn_final.png',bbox_inches='tight',format='png')

### Make figures of a full shot gather to be used in presentations and publications

In [ ]:
xcut=200

p=0.02
vmax=p*np.amax(praw)

fig=plt.figure(figsize=(6, 4),dpi=300)
plt.imshow(np.transpose(praw[xcut:-xcut,:]),interpolation='sinc',aspect="auto",extent=[-800+xcut,800-xcut,0.35,0],cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.savefig(figpath+'raw_das_example.png',bbox_inches='tight',format='png')

In [ ]:
fig=plt.figure(figsize=(6.66, 5),dpi=300)


plt.subplot(2,2,1)
p=0.05
vmax=p*np.amax(proc)
plt.imshow(np.transpose(proc/vmax),interpolation='sinc',aspect="auto",extent=[-650,650,0.35,0],cmap='Greys',vmin=-1,vmax=1)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.colorbar(format="%.1f",pad=0.01,shrink=0.5)
plt.title(r"a) t-x domain",loc='left')

plt.subplot(2,2,2)
p=0.2
vmax=p*np.amax(proc_fx)
plt.imshow(np.transpose(proc_fx/vmax),interpolation='bilinear',aspect="auto",extent=[-650,650,1000,0],cmap='jet',vmin=0,vmax=1)
plt.xlabel('Offset (m)')
plt.ylabel('Frequency (Hz)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.gca().invert_yaxis()
plt.colorbar(format="%.1f",pad=0.01,shrink=0.5)
plt.title(r"b) f-x domain",loc='left')

plt.subplot(2,2,3)
p=0.2
vmax=p*np.amax(proc_fk)
gl=10.0
plt.imshow(np.transpose(proc_fk/vmax),interpolation='bilinear',aspect="auto",extent=[-math.pi,math.pi,1000,0],cmap='jet',vmin=0,vmax=1)
plt.vlines([-2*math.pi/gl,2*math.pi/gl],0,1000,linestyles='--',colors='k',linewidth=0.75)
plt.xlabel('Wavenumber (1/m)')
plt.ylabel('Frequency (Hz)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.gca().invert_yaxis()
plt.colorbar(format="%.1f",pad=0.01,shrink=0.5)
plt.title(r"c) f-k domain",loc='left')

plt.subplot(2,2,4)
p=0.4
vmax=p*np.amax(proc_fv)
gl=10.0
plt.imshow(np.transpose(proc_fv/vmax),interpolation='bilinear',aspect="auto",extent=[0,1000,6.5,-6.5],cmap='jet',vmin=0,vmax=1)
plt.plot([0,1000],[0,gl], 'k--', linewidth=0.75)
plt.plot([0,1000],[0,-gl], 'k--',linewidth=0.75)
plt.ylim([6.5,-6.5])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase velocity (km/s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.gca().invert_yaxis()
plt.colorbar(format="%.1f",pad=0.01,shrink=0.5)
plt.annotate("P", xy=[250,4.5],xytext=[350,5.0],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SH",xy=[170,3.0],xytext=[350,2.8],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SV",xy=[160,2.0],xytext=[300,2.0],color='w', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.title(r"d) v-f domain",loc='left')

plt.tight_layout()

plt.savefig(figpath+'proc_das_example.png',bbox_inches='tight',format='png')

In [ ]:
# plot the strong horizontal noise after stacking to estimate the original source duration
asum = np.sum(praw[1000:,:100],axis=0)
asum = asum - np.mean(asum)

In [ ]:
plt.plot(asum)
plt.show()